In [1]:
# !pip install nltk    
# !pip install emoji        
# !pip install autocorrec\t    
# !pip install xgboost
# !pip install -U spacy

In [2]:
# nltk.download('punkt')  # for using word_tokenizer
# nltk.download('wordnet')  # for using Lemmatizer
# nltk.download('averaged_perceptron_tagger') # for language processing i.e tagging words with their parts of speech (POS)

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import spacy
import nltk
import re   # regular expression
import string
import emoji
from autocorrect import Speller   # for correcting spelling
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize    # for tokenizing string into words
from nltk.stem import WordNetLemmatizer    # for lemmatizing words
from nltk.tag import pos_tag # for tagging words with their parts of speech (POS)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2, mutual_info_classif,f_classif

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost
from xgboost import XGBClassifier

In [16]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score

In [18]:
df_prepro_train = pd.read_csv('processed_train.csv')

In [19]:
print(df_prepro_train.shape)
df_prepro_train.head()

(808661, 6)


,ID,Comments,Parent Comments,Text,Pre Processed Text,Class Labels
0,ocxtitan,Central Illinois,Jesus; where do you live?,Jesus; where do you live? Central Illinois,jesus live central illinois,non-sarcastic
1,LeChuckly,To think - CNN used to be the acronym synonymo...,Even The CNN Staff Is Sick Of The Wall-To-Wall...,Even The CNN Staff Is Sick Of The Wall-To-Wall...,even cnn staff sick walltowall trump coverage ...,non-sarcastic
2,throwitskrub8,But then again; you have to consider that all ...,agree to that part.It can also mean that gujra...,agree to that part.It can also mean that gujra...,agree parti also mean gujarat husbands good su...,non-sarcastic
3,fresherthanyouuu,ughhhhh,If a guy told you he doesn't use social media ...,If a guy told you he doesn't use social media ...,guy tell use social media go head ughhhhh,non-sarcastic
4,_kushagra,I should've put the,No; it's just a programming bug. After all; th...,No; it's just a programming bug. After all; th...,program bug android app music service still be...,sarcastic


In [20]:
df_prepro_train.isna().sum()

ID                     0
Comments              38
Parent Comments        0
Text                   0
Pre Processed Text     0
Class Labels           0
dtype: int64

In [21]:
# test
df_prepro_test = pd.read_csv('processed_test.csv')
print(df_prepro_test.shape)
df_prepro_test.head()

(202166, 5)


,ID,Comments,Parent Comments,Text,Pre Processed Text
0,theyoungthaddeus,"No one ""needs"" an assault foam dart blaster",Your son has to register those at the county j...,Your son has to register those at the county j...,son register county jungle gym mags hold ounce...
1,Just_an_asian_here,Cause all attractive women are uninteresting a...,Likely due to creative and interesting content.,Likely due to creative and interesting content...,likely due creative interesting content cause ...
2,Foxprowl,Poser.,Jon Stewart is going to HBO,Jon Stewart is going to HBO Poser.,jon stewart go hbo power
3,kd7rzv,Won't be long and Anet will start banning peop...,This post looks like bullshit market manipulat...,This post looks like bullshit market manipulat...,post looks like bullshit market manipulation w...
4,Ellefied,There goes my hope that Kubo does a Kojima as ...,Plus the Japanese typically do not talk shit w...,Plus the Japanese typically do not talk shit w...,plus japanese typically talk shit come busines...


In [22]:
df_prepro_test.isna().sum()

ID                     0
Comments              15
Parent Comments        0
Text                   0
Pre Processed Text     0
dtype: int64

In [23]:
# X_tst.isna().sum() # problem: If the sentence is madeup of only stopwords, numbers etc., the sentence after preprocess becomes empty results in NaN
# # fix, after preprocess check sentence length if its 0, return the orig sentence without preprocess. else preprocess. 

In [24]:
# # df[['Comments','Parent Comments']][df['Comments'].isnull()]
# X_tst[X_tst['Text'].isnull()]

In [25]:
# Train data
X = df_prepro_train['Pre Processed Text']
y_trn = df_prepro_train['Class Labels']

# Test data
X_test = df_prepro_test['Pre Processed Text']

In [26]:
# converting Labels coloumn into numerical form of train data
encoder = LabelEncoder()
y = encoder.fit_transform(y_trn)

In [27]:
encoder.classes_

array(['non-sarcastic', 'sarcastic'], dtype=object)

In [28]:
np.unique(y)

array([0, 1])

In [29]:
# train validation split of data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=3,stratify=y)
X_train.shape,X_valid.shape

((646928,), (161733,))

# BoW

In [30]:
# X1 = np.hstack(X.values) # since Count vec takes only list of items
# X_tst1 = np.hstack(X_tst.values)

In [31]:
cv = CountVectorizer(ngram_range=(1,3),max_features=5000) # (1,3) means Unigrams, Bigrams and Trigrams
X_train_bow = cv.fit_transform(X_train).toarray()
X_valid_bow = cv.transform(X_valid).toarray()
X_test_bow = cv.transform(X_test).toarray()

X_train_bow.shape,X_valid_bow.shape,X_test_bow.shape

((646928, 5000), (161733, 5000), (202166, 5000))

In [32]:
# np.savez_compressed("X_train_bow.npz", X_train_bow)

# loaded_array = np.load("X_train_bow.npy")
# print(loaded_array)

In [ ]:

# # convert array into dataframe
# dbtr = pd.DataFrame(X_train_bow)
# dbvd = pd.DataFrame(X_valid_bow)
# dbtt = pd.DataFrame(X_test_bow)

# # save the dataframe as a csv file
# dbtr.to_csv("x_train_bow.csv",index=False)
# dbvd.to_csv("x_valid_bow.csv",index=False)
# dbtt.to_csv("x_test_bow.csv",index=False)

# tf-idf

In [18]:
tfidf = TfidfVectorizer(ngram_range=(1,3),max_features=60000,use_idf=True,smooth_idf=True)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_valid_tfidf = tfidf.transform(X_valid).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

X_train_tfidf.shape,X_valid_tfidf.shape,X_test_tfidf.shape

((646928, 60000), (161733, 60000), (202166, 60000))

In [ ]:
# # convert array into dataframe
# dbbtr = pd.DataFrame(X_train_tfidf)
# dbbvd = pd.DataFrame(X_valid_tfidf)
# dbbtt = pd.DataFrame(X_test_tfidf)

# # save the dataframe as a csv file
# dbbtr.to_csv("x_train_tfidf.csv",index=False)
# dbbvd.to_csv("x_valid_tfidf.csv",index=False)
# dbbtt.to_csv("x_test_tfidf.csv",index=False)

# word embedding

### Word2Vec

In [ ]:
# !python -m spacy download en_core_web_lg

In [182]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
X_train,X_valid,X_test_bow

NameError: name 'X_test_bow' is not defined

In [79]:
X_train

789798    drake nba millions benjamin franklin canadian ...
667532    marijuana marijuana quite popularity history a...
736283    separate oxygen hydrogen water ignite well wal...
176253         things go bump night allow take picture lego
801665    girlfriends birthday gift show us picture unde...
                                ...                        
114413    happen america september day change history im...
77202     meth maybe sometimes method friday saturday mo...
471315    extra chief trump responsible us attack skips ...
748699    common times actually period time jewish troop...
306343    people seem forget pence ultra right wing chri...
Name: Pre Processed Text, Length: 646928, dtype: object

In [80]:
class MySentences(object):
    def __init__(self, sentences):
        self.sentences = sentences 
    def __iter__(self):
        for line in self.sentences:
            yield line.split()

In [183]:
input_arr = []
for item in X_train.values:
    doc = nlp(item)
    input_arr.append(doc.vector)
    break

In [187]:
# input_arr
# X_train.values

In [82]:
from gensim.models import Word2Vec

In [83]:
sentences = MySentences(X_train)

In [84]:
for s in sentences:
    print(s)
    break

['drake', 'nba', 'millions', 'benjamin', 'franklin', 'canadian', 'dollars']


In [85]:
model = Word2Vec(sentences=sentences,vector_size=100,sg= 0,window=5,min_count=1,workers=32)

In [86]:
model.save('word2vec.model')

In [87]:
model.corpus_total_words

11633611

In [88]:
print(model)

Word2Vec<vocab=144163, vector_size=100, alpha=0.025>


In [102]:
model.wv['computer','laptop']

array([[ 1.4755176e+00, -1.0003923e+00, -5.7706565e-01, -1.9328494e-01,
         7.7569312e-01,  2.2920718e+00,  2.2325490e+00,  7.3089516e-01,
         2.3061707e+00,  1.8381729e+00,  1.4842108e+00, -1.6856040e+00,
        -3.4607871e+00, -3.5753337e-01, -4.0258390e-01,  1.3328685e+00,
        -1.1337510e+00, -6.2122124e-01,  3.1225592e-01, -6.1956781e-01,
         6.5621489e-01,  1.1669500e+00, -2.8425914e-01,  5.1055533e-01,
        -9.3304992e-01, -1.0259293e+00,  8.0554563e-01, -1.0876219e+00,
        -1.0551823e+00, -1.0465925e+00, -2.3096409e+00,  9.1883606e-01,
         1.2544596e+00, -1.6090335e-01,  1.3296086e+00, -7.8464472e-01,
        -8.2915300e-01,  1.1054187e+00, -1.3686762e+00,  9.8261929e-01,
         1.2752582e+00, -1.3362261e+00,  9.9796218e-01,  1.8851825e+00,
        -1.8528808e+00, -8.8108480e-01,  7.7025574e-01, -6.9624591e-01,
         5.7853645e-01,  2.4737980e+00,  1.0349784e+00, -1.3738530e+00,
        -3.2401863e-01,  1.7385583e+00, -8.5409081e-01,  1.19274

In [91]:
# def word2vec(x):
#     doc = nlp(x)
#     vec = doc.vector
#     return vec

In [92]:
# df['vec'] = df['review'].apply(lambda x: word2vec(x))

In [52]:
from gensim.test.utils import common_texts

In [53]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [113]:
w2v=dict(zip(model.wv.index_to_key, model.wv.vectors))

In [105]:
from sklearn.naive_bayes import GaussianNB

In [176]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec,dim):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = dim

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec.keys()] or [np.zeros(self.dim)], axis=0) for words in X
        ])

In [177]:
np.mean([w2v[w] for w in ['word','computer'] if w in w2v.keys()],axis=0)

array([ 1.0824815 ,  0.71639675,  0.21218523, -0.34808567,  0.77492297,
        1.4794843 ,  0.6010719 ,  0.22179958,  1.6036116 ,  0.9052475 ,
        0.87729186, -0.38980472, -2.2052183 , -0.5118392 , -0.55851465,
        1.9769785 , -0.7538709 ,  0.6616144 , -0.48489776, -1.002662  ,
        1.525203  , -0.16999859, -0.4207438 ,  1.139093  , -1.515057  ,
        0.30103773,  0.20051995,  0.23532754,  0.20731485, -1.3388798 ,
       -0.90691745,  1.703035  ,  2.0329447 , -0.80370677,  0.09067011,
       -0.32836422, -0.44403347,  0.64649737, -1.2978325 ,  1.3318552 ,
        1.2680016 , -0.3032618 ,  0.18964666,  1.4722872 , -1.2167127 ,
        0.18335038,  0.19206388, -0.9325362 , -0.28691283,  0.9847708 ,
        1.3986429 , -0.6484471 ,  0.3222345 ,  0.6789633 , -0.4748214 ,
        0.70946497, -2.216866  , -0.75549024, -1.1875656 , -0.5204948 ,
       -0.17813279, -0.7590742 ,  1.5861192 ,  0.05278382,  0.38371533,
       -0.05811653, -1.7596617 ,  0.3337974 , -0.8002107 ,  0.35

In [180]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v,100)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200,n_jobs=32))])
# etree_w2v_tfidf = Pipeline([
#     ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
#     ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [181]:
cross_val_score(etree_w2v,X_train,y_train,cv=5).mean()

0.5270200072863425

In [139]:
MeanEmbeddingVectorizer(w2v).transform([['computer','word']])

array([[ 1.0824815 ,  0.71639675,  0.21218523, -0.34808567,  0.77492297,
         1.4794843 ,  0.6010719 ,  0.22179958,  1.6036116 ,  0.9052475 ,
         0.87729186, -0.38980472, -2.2052183 , -0.5118392 , -0.55851465,
         1.9769785 , -0.7538709 ,  0.6616144 , -0.48489776, -1.002662  ,
         1.525203  , -0.16999859, -0.4207438 ,  1.139093  , -1.515057  ,
         0.30103773,  0.20051995,  0.23532754,  0.20731485, -1.3388798 ,
        -0.90691745,  1.703035  ,  2.0329447 , -0.80370677,  0.09067011,
        -0.32836422, -0.44403347,  0.64649737, -1.2978325 ,  1.3318552 ,
         1.2680016 , -0.3032618 ,  0.18964666,  1.4722872 , -1.2167127 ,
         0.18335038,  0.19206388, -0.9325362 , -0.28691283,  0.9847708 ,
         1.3986429 , -0.6484471 ,  0.3222345 ,  0.6789633 , -0.4748214 ,
         0.70946497, -2.216866  , -0.75549024, -1.1875656 , -0.5204948 ,
        -0.17813279, -0.7590742 ,  1.5861192 ,  0.05278382,  0.38371533,
        -0.05811653, -1.7596617 ,  0.3337974 , -0.8

### Glove

In [104]:
gnb = GaussianNB()
gnb.fit(X_train_bow,y_train)
y_pred = gnb.predict(X_test_bow)

NameError: name 'GaussianNB' is not defined

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.5.tar.gz

In [ ]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

In [ ]:
import spacy
import en_core_web_sm
# Load the spacy model.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc = nlp(X_train.values[0])
print(doc.vector)

[ 0.15583833  0.22911808  0.13835862 -0.2090293  -0.00363565 -0.13118638
 -0.15676734  0.23040245 -0.24502586 -0.10665981  0.23412983 -0.21705897
  0.13545002  0.13861737  0.12415285  0.21369664 -0.19687523 -0.10246313
 -0.03955808  0.27656794 -0.11084636 -0.28008527  0.3439968   0.0482447
 -0.06924975 -0.03105986 -0.54718924 -0.3975329   0.1704354  -0.16617411
  0.09593288 -0.31931752  0.42519966  0.05527496  0.06130749 -0.3888249
 -0.14431256  0.07848645  0.07683445  0.3278561  -0.3507611  -0.1357377
  0.28880984  0.09166662  0.0593424  -0.04693977  0.1484201   0.01081875
  0.25666746  0.21095097  0.1744978   0.04549917  0.19118404 -0.25923675
 -0.06680956  0.25583443 -0.06317639 -0.2850735  -0.03264944  0.0042502
 -0.07356709 -0.09716224  0.08560921 -0.09941689 -0.12068396 -0.21283795
 -0.03129313 -0.32813746  0.42847297 -0.37240502  0.26251498 -0.11578684
 -0.03221569 -0.05079779  0.17090753 -0.12238634 -0.07934     0.22784583
  0.20331696 -0.5104732   0.07128731 -0.03987676 -0.05812799  0.19840325
 -0.10885198 -0.09540041  0.2161586  -0.17952771  0.04469092  0.06091264
  0.05722181 -0.02751416  0.19805025  0.32649586 -0.06467736  0.10155083]

In [ ]:
input_arr = np.array(input_arr)

In [ ]:
input_arr.shape

In [ ]:
input_test_arr = []
for item in X_test.values:
    doc = nlp(item)
    input_test_arr.append(doc.vector)

In [ ]:
input_test_arr = np.array(input_test_arr)
input_test_arr.shape

In [103]:
gnb = GaussianNB()
gnb.fit(input_arr,y_train)

NameError: name 'GaussianNB' is not defined

In [ ]:
y_pred = gnb.predict(input_test_arr)
accuracy_score(y_test,y_pred)

# Models

### With Feature Selection

In [16]:
def logistic(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    fsl = 0
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=100))
    
    print('\n\t ---------- Training Logistic Regression Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline1 = Pipeline([('fs',fsl),
                          ('clf1', LogisticRegression(class_weight='balanced'))])
    clf1_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf1__penalty':['none', 'l1', 'l2', 'elasticnet',],
        'clf1__C':[10,0.01,0.001,0.003,0.004],     
        'clf1__solver':['newton-cg','liblinear','sag',]
        }
    grid_search1 = GridSearchCV(estimator=pipeline1, param_grid=clf1_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search1.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search1.best_score_}")
    clf1 = grid_search1.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf1) 
    predicted_class_labels1 = clf1.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels1))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf1

In [82]:
def DecisionTree(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training Decision Tree Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline2 = Pipeline([('fs', fsl),
                          ('clf2', DecisionTreeClassifier(random_state=40))])
    clf2_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf2__criterion':['entropy','gini',], 
        'clf2__max_features':['sqrt', 'log2',None],
        'clf2__max_depth':[10,25,40,100],
        'clf2__ccp_alpha':[0.002,0.004,0.009,0.01,0.1,]
        }
    grid_search2 = GridSearchCV(estimator=pipeline2, param_grid=clf2_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search2.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search2.best_score_}")
    clf2 = grid_search2.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf2)
    predicted_class_labels2 = clf2.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels2))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf2

In [83]:
def KNN(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training KNN Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline3 = Pipeline([('fs', fsl),
                          ('clf3', KNeighborsClassifier())])
    clf3_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf3__n_neighbors': [1,2,3,5,10,20,25,35,36,38,],      
        'clf3__weights':['uniform', 'distance',],
        'clf3__p':[1,2,],
        'clf3__metric':['euclidean', 'manhattan',] 
        }
    grid_search3 = GridSearchCV(estimator=pipeline3, param_grid=clf3_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search3.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search3.best_score_}")
    clf3 = grid_search3.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf3) 
    predicted_class_labels3 = clf3.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels3))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf3

In [84]:
def GaussianNB(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    # Gaussian Naive Bayes
    print('\n\t ---------- Training Gaussian Naive Bayes Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline4 = Pipeline([('fs', fsl),
                          ('clf4', GaussianNB())])
    clf4_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf4__var_smoothing': np.logspace(0,-9, num=60)
        }
    grid_search4 = GridSearchCV(estimator=pipeline4, param_grid=clf4_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search4.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search4.best_score_}")
    clf4 = grid_search4.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf4) 
    predicted_class_labels4 = clf4.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels4))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf4

In [85]:
def MultinomialNB(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training Multinomial Naive Bayes Classifier ---------- \n')  
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline5 = Pipeline([('fs', fsl),
                          ('clf5', MultinomialNB(fit_prior=True, class_prior=None))])
    clf5_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf5__alpha':[0,1,]
        }
    grid_search5 = GridSearchCV(estimator=pipeline5, param_grid=clf5_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search5.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search5.best_score_}")
    clf5 = grid_search5.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf5) 
    predicted_class_labels5 = clf5.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels5))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf5

In [86]:
def LinearSVC(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training Linear SVC Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline8 = Pipeline([('fs', fsl),
                          ('clf8', LinearSVC(class_weight='balanced'))])
    clf8_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf8__C':[0.001,0.01,1,100,],
        }  
    grid_search8 = GridSearchCV(estimator=pipeline8, param_grid=clf8_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search8.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search8.best_score_}")
    clf8 = grid_search8.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf8) 
    predicted_class_labels8 = clf8.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels8))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf8

In [87]:
def SVC(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training SVM Classifier ---------- \n') 
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline9 = Pipeline([('fs', fsl),
                          ('clf9', SVC(probability=True))])
    clf9_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf9__C':[0.1,1,50,60,70,80,100,150],
        'clf9__kernel':['poly','linear','sigmoid','rbf',],
        }
    grid_search9 = GridSearchCV(estimator=pipeline9, param_grid=clf9_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search9.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search9.best_score_}")
    clf9 = grid_search9.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf9) 
    predicted_class_labels9 = clf9.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels9))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf9

In [88]:
def RandomForest(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000)) 
    
    print('\n\t ---------- Training Random Forest Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline10 = Pipeline([('fs', fsl),
                           ('clf10', RandomForestClassifier(class_weight='balanced'))])
    clf10_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf10__criterion':['entropy','gini',],
        'clf10__max_depth':[10,25,30,45,50,80,100,200,],
        'clf10__n_estimators':[30,50,100,],
        'clf10__max_features':['sqrt','log2',None,] 
        } 
    grid_search10 = GridSearchCV(estimator=pipeline10, param_grid=clf10_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search10.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search10.best_score_}")
    clf10 = grid_search10.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf10) 
    predicted_class_labels10 = clf10.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels10))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf10

In [89]:
def AdaBoost(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000)) 
    
    print('\n\t ---------- Training AdaBoost Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    be1 = SVC(kernel='poly',C=100,probability=True)              
    be2 = LogisticRegression(solver='newton-cg',C=0.001)        
    pipeline11 = Pipeline([('fs', fsl),
                           ('clf11', AdaBoostClassifier(algorithm='SAMME.R'))])
    clf11_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf11__base_estimator':[be1,be2,],
        'clf11__n_estimators':[10, 30,50, 80,100, 300,], 
        'clf11__learning_rate':[0.0001, 0.001, 0.01, 0.1,],
        'clf11__random_state':[40,]
        }  
    grid_search11 = GridSearchCV(estimator=pipeline11, param_grid=clf11_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search11.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search11.best_score_}")
    clf11 = grid_search11.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf11) 
    predicted_class_labels11 = clf11.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels11))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf11

In [90]:
def LDA(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training LDA Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline12 = Pipeline([('fs', fsl),
                           ('clf12', LinearDiscriminantAnalysis())])
    clf12_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf12__solver':['svd','lsqr', 'eigen',],
        'clf12__shrinkage':['auto',0.0005,0.0008,0.001,0.005,0.01,0.1,None,]
        } 
    grid_search12 = GridSearchCV(estimator=pipeline12, param_grid=clf12_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search12.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search12.best_score_}")
    clf12 = grid_search12.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf12) 
    predicted_class_labels12 = clf12.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels12)) 
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf12

In [91]:
def XGBoost(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    
    if fs == 'MI' :
        fsl =  SelectKBest(score_func = mutual_info_classif)
    elif fs == 'chi2':
        fsl =  SelectKBest(SelectKBest(chi2, k=1000))
    
    print('\n\t ---------- Training XGBoost Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline13 = Pipeline([('fs', fsl),
                           ('clf13', XGBClassifier(booster = 'gbtree',objective='multi:softmax',num_class = 4,eval_metric = 'merror'))])
    clf13_parameters = {
        'fs__k' : [i+1 for i in range(x_train.shape[1])],
        'clf13__n_estimators':[20,50,80,100,200,], 
        'clf13__max_depth':[5,10,15,30,50,100,], 
        'clf13__learning_rate':[0.15, 0.2, 0.3,],
        'clf13__gamma':[0,0.1,0.2,0.3,],
        'clf13__min_child_weight':[1,3,5,7,30,], 
        'clf13__colsample_bytree':[0.3,0.6,0.9,1,2], 
        'clf13__reg_lambda':[0,0.1,0.2,0.4,0.8,],      
        }
    grid_search13 = GridSearchCV(estimator=pipeline13, param_grid=clf13_parameters, n_jobs=-1, cv=cv, scoring='f1_macro')
    grid_search13.fit(x_train,y_train)
    print(f"Best score on Training set :  {grid_search13.best_score_}")
    clf13 = grid_search13.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf13) 
    predicted_class_labels13 = clf13.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels13))   
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf13

##### bow training

In [35]:
def logisticw(x_t,x_v,y_t, y_v,fs):
    t_start = time.time() # in seconds
    x_train = x_t
    x_valid = x_v
    y_train = y_t
    y_valid = y_v
    fs = fs
    # fsl = 0
    # if fs == 'MI' :
    #     fsl =  SelectKBest(score_func = mutual_info_classif)
    # elif fs == 'chi2':
    # fsl =  SelectKBest(SelectKBest(chi2, k=100))
    
    print('\n\t ---------- Training Logistic Regression Classifier ---------- \n')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    pipeline1 =  LogisticRegression(class_weight='balanced')
    # clf1_parameters = {
    #     'fs__k' : [i+1 for i in range(x_train.shape[1])],
    #     'clf1__penalty':['none', 'l1', 'l2', 'elasticnet',],
    #     'clf1__C':[10,0.01,0.001,0.003,0.004],     
    #     'clf1__solver':['newton-cg','liblinear','sag',]
    #     }
    # grid_search1 = GridSearchCV(estimator=pipeline1, param_grid={}, n_jobs=-1, cv=cv, scoring='f1_macro')
    pipeline1.fit(x_train,y_train)
    
    
    
    print(f"Best score on Training set :  {grid_search1.best_score_}")
    clf1 = grid_search1.best_estimator_
    print('\n\n The best set of parameters of the pipeline in Training Phase are: ')
    print(clf1) 
    predicted_class_labels1 = clf1.predict(x_valid)  # validation

    print('\n *******  Scores on Validation Data  ******* \n ')
    print(classification_report(y_valid, predicted_class_labels1))
    
    t_ends = time.time() # in seconds
    t_net = (t_ends - t_start)/60    # in minutes
    net_time = round(t_net,2)
    print("====================================================================")
    print(f"Process Completed and time taken is : {net_time} minutes")
    print("====================================================================")
    
    return clf1

In [35]:
m = LogisticRegression(n_jobs=32,max_iter=10)
# k = Se
m.fit(X_train_bow,y_train)
# clf1 = logisticw(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

KeyboardInterrupt: 

In [80]:
clf2 = DecisionTree(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf3 = KNN(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf4 = GaussianNB(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf5 = MultinomialNB(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf8 = LinearSVC(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf9 = SVC(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf10 = RandomForest(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf11 = AdaBoost(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf12 = LDA(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
clf13 = XGBoost(X_train_bow, X_valid_bow, y_train, y_valid,'chi2')

In [ ]:
# Evaluation
print('\n Total documents in the training set: '+str(len(trn_data))+'\n')    
print('\n Total documents in the test set: '+str(len(tst_data))+'\n') 

pr=precision_score(tst_cat, predicted, average='binary') 
print ('\n Precision:'+str(pr)) 

rl=recall_score(tst_cat, predicted, average='binary') 
print ('\n Recall:'+str(rl))

In [ ]:
clf1_mis = logistic_fs(x_trains,x_valids,y_train, y_valid,'MI') # by standardisation

In [ ]:
clf1_anovas = logistic_fs(x_trains,x_valids,y_train, y_valid,'ANOVA') # Standardisation

In [ ]:
# to get name of the 5 best features in decresing order i.e 1st feature is high import. then 2nd fet then 3rd so on
X_train.columns[sel_five_cols.get_support()] # x_train is the same as my x_train

In [ ]:
# best model

In [ ]:
ffl = logistic_fsf(x_trains,x_valids,y_train,y_valid)

In [ ]:
predl = ffl.predict(x_tests)

In [ ]:
# saving predicted class labels as txt file
np.savetxt('Akash_Singh_test_class_labels.txt',predl,fmt='%d',delimiter='\n')